In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install scipy==1.7.0
import scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 28.5 MB 1.6 MB/s 
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
Predicted = pd.read_csv("./gdrive/MyDrive/ST_Project/SimulatedData/output_Seurat.csv")
Target=pd.read_csv("./gdrive/MyDrive/ST_Project/SimulatedData/target.csv")


In [6]:
Predicted=Predicted.drop(columns="Unnamed: 0")
Target=Target.drop(columns="Unnamed: 0")
Predicted=Predicted.to_numpy()
Target=Target.to_numpy()

# Pearson Correlation
All at once:

In [8]:
prs=(np.corrcoef(Predicted,Target,rowvar=False))
prs=pd.DataFrame(prs)

In [9]:
prs

,0,1,2,3,4,5,6,7,8,9
0,1.000000,-0.391862,-0.394430,-0.297442,-0.328501,0.848878,-0.246584,-0.332657,-0.118116,-0.046507
1,-0.391862,1.000000,0.126081,-0.312494,0.075146,-0.433548,0.677736,0.156365,-0.322491,-0.156958
2,-0.394430,0.126081,1.000000,-0.305173,-0.294131,-0.517140,-0.089721,0.873447,-0.426649,-0.311805
3,-0.297442,-0.312494,-0.305173,1.000000,-0.216189,-0.079400,-0.041292,-0.253711,0.781028,-0.082862
4,-0.328501,0.075146,-0.294131,-0.216189,1.000000,-0.209688,0.016809,-0.321834,-0.061880,0.627545
5,0.848878,-0.433548,-0.517140,-0.079400,-0.209688,1.000000,-0.237878,-0.475712,0.028090,-0.099725
6,-0.246584,0.677736,-0.089721,-0.041292,0.016809,-0.237878,1.000000,-0.142213,-0.174821,-0.306100
7,-0.332657,0.156365,0.873447,-0.253711,-0.321834,-0.475712,-0.142213,1.000000,-0.388622,-0.493745
8,-0.118116,-0.322491,-0.426649,0.781028,-0.061880,0.028090,-0.174821,-0.388622,1.000000,-0.039719
9,-0.046507,-0.156958,-0.311805,-0.082862,0.627545,-0.099725,-0.306100,-0.493745,-0.039719,1.000000


One at a time:

In [10]:
num_celltypes=5
d = np.zeros(num_celltypes)

In [11]:
for i in range(0,num_celltypes):
  p=Predicted[:,i]
  t=Target[:,i]
  prs=(np.corrcoef(p,t,rowvar=False))
  d[i]=prs[0,1]


In [12]:
d

array([0.84887829, 0.67773616, 0.87344658, 0.7810277 , 0.62754486])

# Jensen Shannon

In [13]:
from scipy.spatial import distance

In [14]:
jns=distance.jensenshannon(Predicted,Target,axis=1)

In [15]:
np.mean(jns)

0.41936876756054386

In [16]:
jns=pd.DataFrame(jns)

In [17]:
jns.to_csv("jns.csv")

# F1, Precision, Recall

In [18]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [19]:
thresholds={0.001,0.01,0.1,0.2,0.3}

In [20]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds


In [21]:
for t in thresholds:
  Predicted_logical=(Predicted>t).astype(int)
  Target_logical=(Target>t).astype(int)
  d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
  d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
  d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [22]:
d

,Precision,Recall,F1-score
0.100,0.896371,0.443801,0.593671
0.300,0.717622,0.746961,0.731998
0.200,0.835828,0.592900,0.693711
0.010,0.999316,0.369747,0.539776
0.001,1.000000,0.367375,0.537343
